In [173]:
import math
import matplotlib.pyplot as plt

class Value:
    def __init__(self, data, _children=(), _op='', label = ''):
        self.data = data
        self._prev = _children
        self.grad = 0.0
        self._op = _op
        self._label = label
        self._backward = lambda: None

    def __repr__(self):
        return f'Value: {self.data}'
    
    def __add__(self, other):
        self = self if isinstance(self, Value) else Value(self)
        other = other if isinstance(other, Value) else Value(other)
        
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        self._backward = _backward
        return out

    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        return self + (-other)

    def __mul__(self, other):
        self = self if isinstance(self, Value) else Value(self)
        other = other if isinstance(other, Value) else Value(other)

        out =  Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        self._backward = _backward
        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float))
        out = Value(self.data**other, (self, other), f'**{other}')

        def _backward():
            self.grad += other*self.data**(other-1) * out.grad
        self._backward = _backward

        return out

    def __truediv__(self, other):
        return self * other**-1

    def tanh(self):
        self = self if isinstance(self, Value) else Value(self)
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t, (self, ), 'tanh')

        def _backward():
            self.grad += 1 - self.tanh()**2
        self._backward = _backward
        return out

    def exp(self):
        self = self if isinstance(self, Value) else Value(self)

        out = Value(math.exp(self.data), (self,), 'exp')

        def _backward():
            self.grad += out.data * out.grad

        self._backward = _backward

        return out

    def backprop(node):
        # Topo sort
        topo = []
        visited = set()
        def build_topo(v):
            for child in v._prev:
                if child not in visited:
                    visited.add(child)
                    build_topo(child)
            topo.append(v)
        build_topo(node)
        for node in reversed(topo):
            node._backward()

        for node in reversed(topo):
            print(f'Label: {node._label}, value: {node.data}, grad: {node.grad}')

    def zero_grad(self):
        if not self._prev:
            return 
        
        self.grad = 0
        for node in self._prev:
            node.zero_grad()

a = Value(3, label='a')
b = Value(4, label='b')
c = Value(-2, label='c')
d = a + 1
e = d + a
f = e + 4
L = f * c
d._label = 'd'
e._label = 'e'
f._label = 'f'
L._label = 'L'
L.grad = 1




In [174]:
a = Value(2.0)
b = Value(3.0)
a - b

Value: -1.0

In [164]:
L.zero_grad()
L.grad = 1
L.backprop()
a.data += 0.01 * a.grad
b.data += 0.01 * b.grad
c.data += 0.01 * c.grad
d.data += 0.01 * d.grad
e.data += 0.01 * e.grad
f.data += 0.01 * f.grad
print("L_data: ", f.data * c.data)

Label: L, value: -22, grad: 1
Label: c, value: 29338.78629871642, grad: 138287.66515947928
Label: f, value: 6224.852479544503, grad: 29338.78629871642
Label: , value: 4, grad: 650724.0342531669
Label: e, value: 6220.852479544503, grad: 29338.78629871642
Label: d, value: 6217.852479544503, grad: 29338.78629871642
Label: , value: 1, grad: 650724.0342531669
Label: a, value: 272992.33031895856, grad: 1301448.068506333
L_data:  200251182.83237898


In [582]:
print(d.grad)

-2.0
